# 导入工具包

In [35]:
import random
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

# 设置参数

In [36]:
set_gelu('tanh')  # 切换gelu版本

In [37]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = '../model/albert_small_zh_google/albert_model.ckpt'
dict_path = '../model/albert_small_zh_google/vocab.txt'

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 定义模型

In [38]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

In [39]:
output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    2704384     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
____________________________________________________________________________________________

In [40]:
# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
# AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  # 用足够小的学习率
#     optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
#         1000: 1,
#         2000: 0.1
#     }),
    metrics=['accuracy'],
)

# 生成数据

In [41]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    df_train_data = pd.read_csv("../data/train.csv")
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [42]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

In [43]:
valid_data

[(6,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t已经整改',
  1),
 (8,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t2、安全疏散通道、疏散指示标志、应急照明和安全出口情况；\t已整改',
  1),
 (10, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t4、灭火器材配置及有效情况；\t气压不够，已更换', 0),
 (11,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t通道有堆放货物',
  0),
 (13,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t1、消防安全重点单位应当进行每日防火巡查，并确定巡查的人员、内容、部位和频次。其他单位可以根据需要组织防火巡查。巡查的内容应当包括：1、用火、用电有无违章情况；\t三楼车间天花板脱落，找人维修',
  0),
 (14,
  '工业/危化品类（现场）—2016版\t（三）机械设备安全防护\t1、危险机械设备\t1、冲、剪、压机械、木工机械、砂轮机、注塑机械、压痕机、粉碎机、抛光机等危险设备的安全防护装置完好齐全。\t车间一推台锯未安装防护罩',
  0),
 (20,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t3、消防设施、器材和消防安全标志是否在位、完整；\t走廊通道有一个灭火器到期，立即整改',
  0),
 (24,
  '工业/危化品类（基础）—2016版\t（七）应急管理\t2、应急演练\t1、定期组织应急演练。\t未组织员工进行应急逃生培训                         整改措施：组织员工进行安全消防、应急逃生、消防器材的实际使用培训并做记录。',
  0),
 (26,
  '商贸服务教文卫类（现场）—2016版\t（二）电气安全\t3、电气线路及电源插头插座\t3、电源插座、电源插头应按规定正确接线。\t电源插头应按规定正确接线',
  1),
 (27,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、

In [44]:
len(train_data)

8428

In [45]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [46]:
# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

In [47]:
valid_data

[(6,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t已经整改',
  1),
 (8,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t2、安全疏散通道、疏散指示标志、应急照明和安全出口情况；\t已整改',
  1),
 (10, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t4、灭火器材配置及有效情况；\t气压不够，已更换', 0),
 (11,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t通道有堆放货物',
  0),
 (13,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t1、消防安全重点单位应当进行每日防火巡查，并确定巡查的人员、内容、部位和频次。其他单位可以根据需要组织防火巡查。巡查的内容应当包括：1、用火、用电有无违章情况；\t三楼车间天花板脱落，找人维修',
  0),
 (14,
  '工业/危化品类（现场）—2016版\t（三）机械设备安全防护\t1、危险机械设备\t1、冲、剪、压机械、木工机械、砂轮机、注塑机械、压痕机、粉碎机、抛光机等危险设备的安全防护装置完好齐全。\t车间一推台锯未安装防护罩',
  0),
 (20,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t3、消防设施、器材和消防安全标志是否在位、完整；\t走廊通道有一个灭火器到期，立即整改',
  0),
 (24,
  '工业/危化品类（基础）—2016版\t（七）应急管理\t2、应急演练\t1、定期组织应急演练。\t未组织员工进行应急逃生培训                         整改措施：组织员工进行安全消防、应急逃生、消防器材的实际使用培训并做记录。',
  0),
 (26,
  '商贸服务教文卫类（现场）—2016版\t（二）电气安全\t3、电气线路及电源插头插座\t3、电源插座、电源插头应按规定正确接线。\t电源插头应按规定正确接线',
  1),
 (27,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、

# 验证和预测

In [49]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

In [50]:
class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
#         test_acc = evaluate(test_generator)
#         print(
#             u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
#             (val_acc, self.best_val_acc, test_acc)
#         )
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

In [51]:
def data_pred(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = model.predict([token_ids, segment_ids]).argmax(axis=1)
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

In [52]:
def data_pred2(test_data):
    id_ids, y_pred_ids = [], []
    batch_token_ids, batch_segment_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        batch_token_ids.append(token_ids)
        batch_segment_ids.append(batch_segment_ids)
        id_ids.append(id)
    token_ids = sequence_padding(batch_token_ids)
    segment_ids = sequence_padding(batch_segment_ids)
    y_pred_ids = model.predict([token_ids, segment_ids]).argmax(axis=1)
    return id_ids, y_pred_ids

In [53]:
## 修改代码

In [54]:
def data_pred3(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = int(model.predict([token_ids, segment_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

In [55]:
def data_pred4(test_data):
    id_ids, y_pred_ids = [], []
    batch_token_ids, batch_segment_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        batch_token_ids.append(token_ids)
        batch_segment_ids.append(batch_segment_ids)
        id_ids.append(id)
    token_ids = sequence_padding(batch_token_ids)
    segment_ids = sequence_padding(batch_segment_ids)
    y_pred_ids = model.predict([token_ids, segment_ids]).argmax(axis=1)[:,0]
    return id_ids, list(y_pred_ids)

# 训练和验证

In [56]:
evaluator = Evaluator()

In [57]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=2,
        callbacks=[evaluator]
    )

Epoch 1/2
264/264 [==============================] - 452s 2s/step - loss: 0.2158 - accuracy: 0.9283
val_acc: 0.96920, best_val_acc: 0.96920, test_acc: 0.96920

Epoch 2/2
264/264 [==============================] - 412s 2s/step - loss: 0.0872 - accuracy: 0.9740
val_acc: 0.97452, best_val_acc: 0.97452, test_acc: 0.97452



In [58]:
model.load_weights('best_model.weights')
# print(u'final test acc: %05f\n' % (evaluate(test_generator)))
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.974524



In [59]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.981490



# 模型预测保存结果

In [ ]:
id_ids, y_pred_ids = data_pred3(test_data)

In [ ]:
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

In [ ]:
df_save.to_csv('result.csv')